<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex13noteC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex13noteC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
## 主成分分析の実験
----



---
### 実験1: 外食への年間支出金額データの主成分分析

独立行政法人統計センターが作成している [教育用標準データセット(SSDSE)](https://www.nstac.go.jp/use/literacy/ssdse/) の中の「SSDSE-家計消費」というデータを使って，主成分分析の実験をやってみましょう．

「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files//kaisetsu-C-2022.pdf

この notebook の作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

#### データの入手と前処理

SSDSEのサイトから CSV ファイルを入手して，扱いやすいように加工します．



In [ ]:
URL = 'https://www.nstac.go.jp/sys/files//SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換

# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df.iloc[:, :3]
print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df = df.drop(['Prefecture', 'City'], axis=1)
df = df.set_index('SSDSE-C-2022')

df = df.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df = df.astype('float32') # 値を浮動小数点型に変換

print()
df.head(5)

ここでは，各種の「外食」での支出金額のみを特徴として取り出します．13個の特徴量があります．

In [ ]:
# 一人あたりの外食支出金額のデータを作成
df_feature = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df.columns:
    if c.startswith('LB12') and c != 'LB12':
        print(c, code[c])
        columns[c] = code[c]
        df_feature[c] = df[c]

# 列名をコードから項目名に変更
df_feature = df_feature.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df_feature = df_feature.divide(df['LA03'], axis=0)

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

df_feature.head(5)

---
#### 主成分分析して可視化してみよう



平均を 0 にしたデータ行列 X を作ります．データの次元数 D = 13, データ数 N = 47 です．

In [ ]:
# 平均を 0 にしたデータ行列 X を作る
X = df_feature.to_numpy()
Xm = np.mean(X, axis=0)
X -= Xm
N, D = X.shape
print(X.shape)
print(X[:5, :]) # 最初の5件の特徴量を表示

X の分散共分散行列の固有値固有ベクトルを求めます．

In [ ]:
# X の分散共分散行列の固有値と固有ベクトルを求める
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
print('固有値:', eval)
U = Vt

# 2つの固有ベクトルの値を表示
df_hoge = pd.DataFrame(index=df_feature.columns)
df_hoge['u1'] = U[0]
df_hoge['u2'] = U[1]
df_hoge

固有値の大きい方の二つの固有ベクトルの値は，上記のようになっています．

- u1 の方向は，正の値の項目と負の値の項目がおよそ半々となっています．正の値の項目への支出が平均より多く，負の値の項目への支出が平均より少ない県では，y1 の値が大きくなります．負の値の中では「他の主食的外食」の絶対値が大きいため，この項目の影響が大きそうです．
- u2 の方向は，全ての項目の値が負です．したがって，y2 の値は，外食全体への支出が少ない県で大きくなります．

これら2つの固有ベクトルを使って D = 13 次元のデータを 2 次元に変換してみましょう．

In [ ]:
# 2次元へ次元削減
Y = X @ U[:2, :].T
print(Y.shape)
print(Y[:5, :]) # 最初の5件の特徴量を表示

数値を見てもわかりにくいので，散布図にしてみます．

In [ ]:
# グラフを描く

kinki5 = np.array([n in range(24, 29) for n in range(N)]) # 滋賀，京都，大阪，兵庫，奈良
tohoku = np.array([n in range(1, 7) for n in range(N)]) # 青森，岩手，宮城，秋田，山形，福島

fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[kinki5, 0], Y[kinki5, 1], label='kinki5')
ax.scatter(Y[tohoku, 0], Y[tohoku, 1], label='tohoku')
ax.axvline(0, linestyle='-', color='gray')
ax.axhline(0, linestyle='-', color='gray')
ax.set_xlim(-15000, 15000)
ax.set_ylim(-15000, 15000)
ax.set_aspect('equal')
ax.legend()
for n in range(N):
    plt.annotate(f'{n}', (Y[n, 0]+200, Y[n, 1]+200))
plt.show()

#### ★★★ やってみよう ★★★

上記の結果を見て，次のことを考えよう．結果を紙媒体にメモしておこう．

- 東北6県は
    - 外食に使う金額が多い方？それとも少ない方？
    - うどん，そば，和食，洋食，焼き肉などへの支出が多い方？
- 近畿5府県はどんな傾向？


---
### 実験2: 猫画像を次元削減して再構成してみよう

主成分分析を利用して，猫の顔画像を次元削減してから再構成する実験をやってみましょう．

#### 準備

画像の可視化用の関数を定義します．

In [ ]:
#####  データの最初の nx x ny 枚を可視化
#
def mosaicImage(dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))
            
    return img


画像データを入手して眺めてみましょう．この実験で使っている猫画像は，他の目的に使ってはいけません．

In [ ]:
# 猫画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/cat131.npz
cat = np.load('cat131.npz')['cat131']
print(cat.shape) # 131枚，ひとつの画像は 64 x 64 = 4096 画素のグレイスケール画像

In [ ]:
print('### 猫画像 ###')
print(f'データ数 x 次元数 = {cat.shape[0]} x {cat.shape[1]}')

# 最初の 16 枚を表示 
img = mosaicImage(cat, 4, 4)
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()  

#### 猫画像の主成分分析

まずは平均 0 のデータ行列を作ります．

In [ ]:
# 平均 0 のデータ行列を作成
Xm = np.mean(cat, axis=0)
X = cat - Xm
N, D = X.shape
print(X.shape)

# 平均を画像として表示
plt.axis('off')
plt.imshow(Xm.reshape(64, 64), cmap = 'gray')
plt.show()  

次に，分散共分散行列の固有値固有ベクトルを求めます．

In [ ]:
# X の分散共分散行列の固有値と固有ベクトルを求める
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
print('固有値:', eval)
U = Vt

# 固有ベクトルを可視化
nx, ny = 5, 2
tmp = np.zeros((nx*ny, D))
for h in range(nx*ny):
    tmp[h, :] = U[h, :]
absmax = np.max(np.abs(tmp))
tmp = tmp/absmax*127 + 128
img = mosaicImage(tmp, nx, ny)
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()  

上記の画像は，固有ベクトルのうち固有値の大きい方から10個を左上から右に向かって並べたものです．固有ベクトルの値には正も負もあるので，正の値が白，0 が灰色，負の値が黒になるようにしてあります．

1つ目の固有ベクトルは，額の部分が白っぽく，鼻筋から顎の下辺りが黒くなっています．したがって，額の部分が平均より明るく，鼻筋から顎の下辺りが平均より暗くなっている猫画像に対して大きな値となります．2つ目以降はも，それぞれ異なる特徴を捉えていることがわかります．

一つの画像を選んで，その変換後の特徴量を出力させてみましょう．
`n` の値を変えると，他の猫も選べます．

In [ ]:
n = 12 # データの番号． 0 から 130 まで

# 元の画像を表示
plt.axis('off')
plt.imshow(cat[n, :].reshape(64, 64), cmap = 'gray')
plt.show()

# 変換して得られる特徴量のうち10個を出力
y = X[n, :] @ U[:10, :].T
y

例えば，顎下の部分に注目すると，そこが黒っぽい猫は1つ目の成分が正で，白っぽい猫は負になっているはずです．猫と固有ベクトルを見比べていろいろ調べてみましょう．

#### 再構成してみよう

次のセルを実行すると，主成分分析で得られた変換行列を利用して，猫画像を $H$ 次元に次元削減してから再構成します．

In [ ]:
#@title 猫画像を次元削減してから再構成
H =  0#@param {type: "number"}
assert 0 <= H and H <= D

Z = np.zeros_like(X) + Xm
if H > 0:
    Y = X @ U[:H, :].T
    Z += Y @ U[:H, :]

nx, ny = 2, 10
tmp = np.zeros((nx*ny, D))
for n in range(10):
    tmp[2*n,   :] = cat[n, :]
    tmp[2*n+1, :] = Z[n, :]
img = mosaicImage(tmp, nx, ny)

print(f'{H}次元での再構成')
plt.figure(figsize=(2, 10))
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()


上記の画像では，学習データのうち10匹の猫について，左に元画像，右に再構成画像が並んでます．`H` の値をいろいろ変えてみましょう．

#### 生成してみよう

次のセルでは，$\mathbf{y} = (y_1,y_2, y_3, y_4, y_5)$ の値を自由に選んで，それを再構成して猫画像を生成することができます．適当に値を選んでみよう．


In [ ]:
#@title 猫画像を生成
y1 =  1000 #@param {type: "slider", min:-2000, max:2000}
y2 = -2000 #@param {type: "slider", min:-2000, max:2000}
y3 = -1200 #@param {type: "slider", min:-2000, max:2000}
y4 =   200 #@param {type: "slider", min:-2000, max:2000}
y5 = -1000 #@param {type: "slider", min:-2000, max:2000}

y = np.array([y1, y2, y3, y4, y5])
z = y @ U[:5, :] + Xm

# 生成した画像を表示
plt.axis('off')
plt.imshow(z.reshape(64, 64), cmap = 'gray')
plt.show()